In [ ]:
!pip install pandas recordlinkage tabulate sklearn py_stringmatching matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.5/926.5 KB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.8/643.8 KB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=3ec5d797c0d0e73c4451beffcdc6218298265df3b318a28791e624d97dd5606b
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
  Created wheel for py_stringmatching: filename=py_stringmatching-0.4.3-cp38-cp38-linux_x86_64.whl size=2683008 sha256=6cdd41d1ec64661dd643bba02602ffd7d698bbbbc39ea22ad4dfc85b378e5b7a
  Stored in directory: /root/.cache/pip/wheels/dc/8e/a6/47bb1099972143b8edd2a29726e1643ef

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import time
from pathlib import Path
import os
import warnings
import numpy as np
from tabulate import tabulate
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import string
import re
from google.colab import drive
drive.mount("/content/gdrive/")
warnings.filterwarnings("ignore")
#folder path
path = "/content/gdrive/Shareddrives/bioenergitics_drive"

Mounted at /content/gdrive/


In [ ]:
# Set global variables sub folders
testing_path= "/testing_data"
time_log_path = "/time_log.csv"
result = "/Colab Notebook/Results"
shimmer_path = "/shimmer_data"

In [ ]:
# Read the data drop nulls
time_log_df = pd.read_csv(path+testing_path+time_log_path)
# time_log_df = time_log_df.dropna()

In [ ]:
# Remove empty records and filter out and clean the time log data
pattern = re.compile("(^[a-z]+_[a-z][0-9]+_[0-9]+_[0-9]+_[0-9]+)")
testing_folder_names = [file for file in os.listdir(path+testing_path) if pattern.match(file)]
list_files = [os.path.join(path+testing_path, file)  for file in os.listdir(path+testing_path) if pattern.match(file)]
time_log_df["Path Name"]=time_log_df["Subject"].str.lower()+"_v"+time_log_df["Visit"].astype(int).astype(str)+"_"+time_log_df["Date"].str.replace(".","_")

In [ ]:
# Generate indexing

def remove_na_values(table):
    start_ind = table.index.tolist()[0]
    start_ind_ls = table[~table["indexing"].isna()].index.tolist()
    if (len(start_ind_ls)>0):
        start_ind = start_ind_ls[0]

    end_ind = table.index.tolist()[-1]
    end_ind_ls = table[~table["indexing"].isna()].index.tolist()
    if (len(end_ind_ls)>0):
        end_ind = end_ind_ls[-1]
    return table.loc[start_ind:end_ind,:]

testing_folder_names = [file for file in os.listdir(path+testing_path) if pattern.match(file)]
for folder in testing_folder_names:
    # Run with one person but not sc_v1_14_10_2022
    if folder !="sc_v1_14_10_2022":
        print(folder)
        if not os.path.exists(path+result+"/"+folder+"/Merge with Cosmed/"):
            os.makedirs(path+result+"/"+folder+"/Merge with Cosmed/")

        for file in os.listdir(path+result+"/"+folder):
            # handle Cosmed file first
            if file =="cosmed_raw.csv":
                cosmed = pd.read_csv(path+result+"/"+folder+"/cosmed_raw.csv")
                cosmed.insert(loc=10, column='indexing',value="")
                cosmed.reset_index(inplace=True, drop=True)
                cosmed["indexing"]=cosmed.index
                # print("Cosmed",cosmed.shape)
                # Pick the first 30
                # cosmed = cosmed[cosmed["indexing"]<=30]

        for file in os.listdir(path+result+"/"+folder):
            if "_Summary.csv" in file:
                summary = pd.read_csv(path+result+"/"+folder+"/"+file)
                summary["Date"]=summary["Time"].str.split(" ").str[0]
                summary["t"]= summary["Time"].str.split(" ").str[1]
                summary["t"]= summary["t"].str[:8]
                # print("Summary",summary.shape)
                summary.drop(columns=["Time","Date"],inplace=True)
                summary.reset_index(inplace=True,drop=True)
                # print(tabulate(summary_gb.head(20), headers=summary_gb.columns.tolist(), tablefmt="pretty"))

                # Cosmed + Zephr
                merge_cos_zephr = summary.merge(cosmed, how='left', on="t")
                merge_cos_zephr= remove_na_values(merge_cos_zephr)
                merge_cos_zephr.reset_index(drop=True,inplace=True)
                merge_cos_zephr.fillna(method='ffill',inplace=True)
                # print("Cosmed + Zephr",merge_cos_zephr.shape)
                merge_cos_zephr.to_csv(path+result+"/"+folder+"/Merge with Cosmed/"+"cos_zeph.csv",index=False)

            elif file=="moxy_data.csv":
                moxy = pd.read_csv(path+result+"/"+folder+"/"+file)
                print("Moxy",moxy.shape)
                moxy.drop(columns=["mm-dd"],inplace=True)
                moxy_gb = moxy.groupby(["hh:mm:ss"]).mean()
                moxy_gb["hh:mm:ss"]=moxy_gb.index
                moxy_gb.reset_index(inplace=True,drop=True)
                # print("Moxy Group",moxy_gb.shape)
                # Cosmed + Moxy
                merge_cos_moxy = moxy_gb.merge(cosmed[["t","indexing"]], how='left', right_on='t',left_on='hh:mm:ss')
                merge_cos_moxy = remove_na_values(merge_cos_moxy)
                merge_cos_moxy.reset_index(drop=True,inplace=True)
                merge_cos_moxy.drop(columns=["t","indexing"],inplace=True)
                # print("Cosmed + Moxy",merge_cos_moxy.shape)
                merge_cos_moxy.to_csv(path+result+"/"+folder+"/Merge with Cosmed/"+"cos_moxy.csv",index=False)
        merge_cos_moxy_zephr = merge_cos_zephr.merge(merge_cos_moxy,how="left",left_on="t",right_on="hh:mm:ss")

        # print("Cos_Zephr")
        # print(tabulate(merge_cos_zephr.head(), headers=merge_cos_zephr.columns.tolist(), tablefmt="pretty"))
        # print("Cos_Moxy")
        # print(tabulate(merge_cos_moxy.head(), headers=merge_cos_moxy.columns.tolist(), tablefmt="pretty"))
        # print("Cos+Moxy+Zephr",merge_cos_moxy_zephr.shape)
        # print("Cos_Moxy_Zephr")
        # print(tabulate(merge_cos_moxy_zephr.head(), headers=merge_cos_moxy_zephr.columns.tolist(), tablefmt="pretty"))
        for file in os.listdir(path+result+"/"+folder):
            if file =="shimmer_data":
                for subfile in os.listdir(path+result+"/"+folder+"/"+file):
                    shimmer = pd.read_csv(path+result+"/"+folder+"/"+file+"/"+subfile)
                    shimmer_timestamp = [i for i in shimmer.columns if "Timestamp" in i]
                    if len(shimmer_timestamp)==1:
                        shimmer["t"]= shimmer[shimmer_timestamp[0]].str.split(" ").str[1]
                        shimmer["t"]= shimmer["t"].str[:8]
                        replace_nan_shimmer = shimmer[shimmer["t"].duplicated()]
                        replace_nan_shimmer["t"] = ""
                        shimmer.update(replace_nan_shimmer)
                        shimmer["t"]=shimmer["t"].replace("",np.nan)
                        # print("Shimmer",shimmer.shape)

                        # Cosmed + Shimmer
                        zhr_shimmer_moxy_cosmed = shimmer.merge(merge_cos_moxy_zephr, how='left', on="t")
                        zhr_shimmer_moxy_cosmed.reset_index(drop=True,inplace=True)
                        zhr_shimmer_moxy_cosmed= remove_na_values(zhr_shimmer_moxy_cosmed)
                        zhr_shimmer_moxy_cosmed.fillna(method='ffill',inplace=True)
                        # print("Cosmed + Shimmer",zhr_shimmer_moxy_cosmed.shape)

                        if not os.path.exists(path+result+"/"+folder+"/Merge with Cosmed/zhr_shimmer_moxy_cosmed/"):
                            os.makedirs(path+result+"/"+folder+"/Merge with Cosmed/zhr_shimmer_moxy_cosmed/")
                        zhr_shimmer_moxy_cosmed.to_csv(path+result+"/"+folder+"/Merge with Cosmed/zhr_shimmer_moxy_cosmed/"+subfile+"_zhr_shimmer_moxy_cosmed.csv",index=False)
                        # print("Shimmer Zepher Moxy Cosmed",zhr_shimmer_moxy_cosmed.shape)

                        # print("All")
                        # print(tabulate(zhr_shimmer_moxy_cosmed.head(), headers=zhr_shimmer_moxy_cosmed.columns.tolist(), tablefmt="pretty"))

        print("")

sg_v1_19_10_2022
Moxy (7955, 7)

cc_v1_21_10_2022
Moxy (8556, 7)

gd_v1_24_10_2022
Moxy (9149, 7)

ck_v1_24_10_2022
Moxy (5101, 7)

cc_v2_25_10_2022
Moxy (7171, 7)

gd_v2_26_10_2022
Moxy (6637, 7)

sg_v2_24_10_2022
Moxy (4898, 7)

ck_v2_26_10_2022
Moxy (7396, 7)



In [ ]:
#Handle time conversion for shimmer files
# time_log_shimmer = time_log_df[time_log_df["Sensor Name"].str.contains("Shimmer")]
# for folder in list_files:
#     for file in os.listdir(folder+shimmer_path):
#         if file.endswith(".csv"):
#             with open(folder+shimmer_path+"/"+file) as f:
#                 delimiter = f.readline()[5:-2]
#             df = pd.read_csv(os.path.join(folder+shimmer_path,file),skiprows=1,header=[0,1],delimiter=delimiter)
#             df.columns = df.columns.map('_'.join)
#             # Remove columns having all NaN values
#             df.dropna(axis=1, how='all',inplace=True)
#             # Find all columns having timestam
#             timestamp_cols = [i for i in df.columns if "Timestamp" in i]
#             # Convert the timestamp columns to time
#             for col in timestamp_cols:
#                 try:
#                     df[col] = df[col].apply(lambda x: datetime.fromtimestamp(x/ 1000))
#                 except TypeError as ex:
#                     print(ex)
#                     print(folder+shimmer_path+"/"+file)
#             person_folder = (folder.split("/")[-1])
#             if not os.path.exists(path+result+"/"+person_folder+shimmer_path):
#                 os.makedirs(path+result+"/"+person_folder+shimmer_path)
#             df.to_csv(path+result+"/"+person_folder+shimmer_path+"/"+file,index=False)

In [ ]:
# # Set UTC for the time count of sensors for COSMED_RAW.xlsx
# cosmed_df = time_log_df[time_log_df["File Name"]=="cosmed_raw.xlsx"]
# def convert_time_earlier_zephr(row):
#     element = datetime.strptime(row,"%d/%m/%Y  %H:%M:%S.%f")
#     result = element-timedelta(hours=1)
#     return str(result)

# def convert_time_earlier_moxy(row):
#     if str(row)=="nan":
#         return str(row)
#     element = datetime.strptime(str(row),"%H:%M:%S")
#     result = element-timedelta(hours=1)
#     return str(result)

# def convert_time(row,timestamp):
#     time_token = str(row).split(":")
#     x =  timestamp+timedelta(minutes=int(time_token[1]),seconds=int(time_token[2]),hours=int(time_token[0]))
#     return x.time()

# # Handle COSMED_RAW, Zephr and moxy_daa.csv files
# for folder in list_files:
#     for file in os.listdir(folder):
#         if file=="cosmed_raw.xlsx":
#             df = pd.read_excel(os.path.join(folder,file))
#             df.dropna(subset=["t"],inplace=True)
#             df = df[df["t"]!="s"]
#             start_time_df = cosmed_df[cosmed_df["Path Name"]==folder.split("/")[-1].replace("2022","22")]
#             # print(folder)
#             # print(cosmed_df)
#             # print(start_time_df.head())
#             if start_time_df.shape[0]>0:
#                 start_time= (start_time_df["Date"].str.replace(".","-")+" "+start_time_df["Start Time"]).tolist()[0]
#                 start_time_token = (start_time.split("-"))
#                 start_time_token[-1] = "20"+start_time_token[-1]
#                 start_time = "-".join(start_time_token)
#                 timestamp = datetime.strptime(start_time, "%d-%m-%Y %H:%M:%S")
#                 new_t = df["t"].apply(lambda x: convert_time(x,timestamp))
#                 df.update(new_t)
#             person_folder = (folder.split("/")[-1])
#             if not os.path.exists(path+result+"/"+person_folder):
#                 os.makedirs(path+result+"/"+person_folder)
#             df.to_csv(path+result+"/"+person_folder+"/"+file.replace("xlsx","csv"),index=False)
#         if file =="zephr":
#             person_folder = (folder.split("/")[-1])
#             for subfile in os.listdir(folder+"/"+file):
#                 if "_Summary.csv" in subfile:
#                     df_zephr = pd.read_csv(folder+"/"+file+"/"+subfile)
#                     df_zephr["Time"]=df_zephr["Time"].apply(lambda x: convert_time_earlier_zephr(x))
#                     if not os.path.exists(path+result+"/"+person_folder):
#                         os.makedirs(path+result+"/"+person_folder)
#                     df_zephr.to_csv(path+result+"/"+person_folder+"/"+subfile,index=False)
#         if file=="moxy_data.csv":
#                 person_folder = (folder.split("/")[-1])
#                 df_moxy = pd.read_csv(folder+"/"+file,skiprows=2,header=[1])
#                 df_moxy["hh:mm:ss"]=df_moxy["hh:mm:ss"].apply(lambda x: convert_time_earlier_moxy(x))
#                 df_moxy["hh:mm:ss"]=df_moxy["hh:mm:ss"].str.split(" ").str[-1]
#                 if not os.path.exists(path+result+"/"+person_folder):
#                     os.makedirs(path+result+"/"+person_folder)
#                 df_moxy.to_csv(path+result+"/"+person_folder+"/"+file,index=False)
